## Generate MDE figures as an alternate to using UMAPs
[PyMDE: Minimum-Distortion Embedding](https://pymde.org/)

In [ ]:
!date

#### import libraries

In [ ]:
import scanpy as sc
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
import scvi

# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#### set notebook variables

In [ ]:
# naming
project = 'aging_phase2'

# directories
wrk_dir = '/labshare/raph/datasets/adrd_neuro/brain_aging/phase2'
quants_dir = f'{wrk_dir}/quants'
figures_dir = f'{wrk_dir}/figures'
sc.settings.figdir = f'{figures_dir}/'

# in files
anndata_file = f'{quants_dir}/{project}.multivi.curated_final.h5ad'

# out files

# variables
DEBUG = False
scvi.settings.seed = 42

### load the final anndata file

In [ ]:
%%time
adata = sc.read_h5ad(anndata_file)
print(adata)
if DEBUG:
    display(adata.obs.sample(5))

### visualize the final curated cell-types

#### existing UMAP

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-talk')
    sc.pl.umap(adata, color=['Cell_type', 'cluster_name'], 
               frameon=False, legend_loc='on data')

#### now with MDE

##### generate the MDE embeding

In [ ]:
%%time
adata.obsm['X_mde'] = scvi.model.utils.mde(adata.obsm['MultiVI_latent'], accelerator='gpu')

##### phase1 cell-types annotated

In [ ]:
figure_file = f'_{project}.mde.curated_prev_celltypes_final.png'
with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-talk')
    sc.pl.embedding(adata, basis='X_mde', color=['Cell_type'], 
                    frameon=False, legend_loc='on data', save=figure_file)

##### phase2 cell-types annotated

In [ ]:
figure_file = f'_{project}.mde.curated_cluster_names_final.png'
with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-talk')
    sc.pl.embedding(adata, basis='X_mde', color=['cluster_name'], 
                    frameon=False, legend_loc='on data', save=figure_file)

In [ ]:
figure_file = f'_{project}.mde.curated_celltype_final.png'
with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-talk')
    sc.pl.embedding(adata, basis='X_mde', color=['curated_type'], 
                    frameon=False, legend_loc='on data', save=figure_file)

In [ ]:
# %%time
# import torch
# import pymde
# pymde.seed(42)
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

# mde = pymde.preserve_neighbors(adata.obsm['MultiVI_latent'], device=device, verbose=True)
# embedding = mde.embed(verbose=True)
# # adata.obsm['X_mde'] = embedding.cpu().numpy()

In [ ]:
# with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
#     plt.style.use('seaborn-v0_8-talk')
#     pymde.plot(embedding, color_by=adata.obs.Cell_type, marker_size=50)
#     plt.show()

In [ ]:
# with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
#     plt.style.use('seaborn-v0_8-talk')
#     pymde.plot(embedding, color_by=adata.obs.cluster_name, marker_size=50)
#     plt.show()

In [ ]:
!date